In [14]:
%%capture --no-stderr

In [9]:
!pip install langchain langgraph langchain_community langchain_openai

In [10]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["TAVILY_API_KEY"] ="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"

In [11]:
!pip install pdfplumber

In [12]:
!pip install langchain_anthropic

In [13]:
!pip install langchain_chroma

In [15]:
!pip install langchain-community pypdf

In [16]:
%pip install -qU "langchain-community>=0.2.11" tavily-python

In [17]:
!pip install faiss-cpu

In [18]:
!pip install google-search-results

In [22]:
pip install sentence-transformers


In [27]:
import os
import getpass
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings


os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your API key: ")
model = chat = llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")



Enter your API key: ··········


In [28]:
from typing import Annotated

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool


In [29]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)



In [ ]:
from typing import Optional
from pydantic import Field, PrivateAttr
from langchain.tools import BaseTool
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.callbacks.manager import CallbackManagerForToolRun
from langchain.tools import tool

from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader


from langchain_core.vectorstores import InMemoryVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter


from typing import Optional
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain_core.callbacks import CallbackManagerForToolRun

class PDFTextExtractorTool(BaseTool):
    name: str = "PDFTextExtractor"
    description: str = "Extracts relevant data from a PDF file located at the specified path. Provide the path and query."

    def _run(self, query: str, pdf_path: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
            # Load the PDF file
            loader = PyPDFLoader(pdf_path)
            docs = loader.load()
            text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=50, chunk_overlap=5, add_start_index=True)
            all_splits = text_splitter.split_documents(docs)
            vector_store = InMemoryVectorStore(embeddings)
            ids = vector_store.add_documents(documents=all_splits)
            results = vector_store.similarity_search(query)
            print(results[0])
            return results[0]


extract_tool = PDFTextExtractorTool()


from langchain_community.tools import TavilySearchResults

tavily_tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
)

from langchain.agents import AgentType, initialize_agent
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain_openai import ChatOpenAI


import os
from langchain_community.utilities import SerpAPIWrapper
os.environ["SERPAPI_API_KEY"] = "f9bec0683ce3cf7380f98d08f08e2b55666b4e37b13f074f663f7f154c480443"

@tool
def google_search(query: str):
    """Search Google for recent and realtime results."""
    search = SerpAPIWrapper()
    return search.run(query)




import os
import requests



def download_pdfs(urls: list, folder_path: str) -> None:
    """
    Downloads PDFs from a given list of URLs and saves them to the specified folder.

    :param urls: List of URLs of the PDF files.
    :param folder_path: The folder where the PDFs should be saved.
    """
    # Ensure the folder exists
    os.makedirs(folder_path, exist_ok=True)

    for index, url in enumerate(urls, start=1):
        try:
            # Extract the filename from the URL or use a default name
            filename = url.split('/')[-1]
            if not filename.endswith('.pdf'):
                filename = f"file_{index}.pdf"  # Fallback filename

            save_path = os.path.join(folder_path, filename)

            # Download and save the PDF
            response = requests.get(url)
            response.raise_for_status()  # Raise an error for HTTP issues
            with open(save_path, 'wb') as file:
                file.write(response.content)
            print(f"PDF successfully downloaded and saved to {save_path}")

        except Exception as e:
            print(f"Failed to download {url}: {e}")


folder_path = "/content/sample_data/pdfs"


from googlesearch import search


@tool
def downloader_tool(query: str, num_results: int = 3):
    """Downloads the online documents from across the internet.

    Args:
        query: The search query.
        num_results: The maximum number of search results to consider (default: 10).
    """
    # 1. Perform a web search for relevant PDFs (limit to top 10)
    search_results = search(f"{query} filetype:pdf", num=num_results)

    # Convert the generator to a list and return the top results
    urls = [url for _, url in zip(range(num_results), search_results)]

    download_pdfs(urls, folder_path)

    print("---")



@tool
def online_retriever_tool():
    """Does nothing.A dummy tool that does nothing but satisfies the tool requirement."""
    return " "












from langgraph.prebuilt import create_react_agent
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain


from langchain import PromptTemplate, LLMChain

consolidation_prompt = PromptTemplate(
    template="""
### Instructions:
**Use Only the Provided Information**:
- Do not source or refer to any additional data.
- Rely exclusively on the content of `information_from_multiple_sources`.

Task: Consolidate knowledge collected from various sources as described:
1. Cluster passages with consistent information.
2. Separate conflicting information into distinct clusters.
3. Exclude irrelevant information.

Context so far: {context}

Consolidated Context after iteration:
""",
    input_variables=["context"],
)

@tool
def iterative_consolidation_tool(initial_context: str, max_iterations: int = 2) -> str:
    """Tool to get consolidated information from initial context."""
    consolidation_chain = LLMChain(llm=llm, prompt=consolidation_prompt)
    context = initial_context

    # Debug: Print the initial context received
    print("------------------")
    # print("\n Initial Context Passed to Tool:\n", context)

    for i in range(max_iterations):
        # Run iterative consolidation
        context = consolidation_chain.run({"context": context})

        # Debug: Print context after each iteration
        print(f" Iteration {i + 1} Context:\n", context)
        print("------------------")

    return context


@tool
def NoOpTool():
    """Does nothing.A dummy tool that does nothing but satisfies the tool requirement."""
    return " "

def make_retriever_prompt(file_path: str, suffix: str) -> str:
    return (
        f"You are a data retrieval agent tasked with extracting relevant information from a file located at '{file_path}'.\n"
        "### Instructions:\n"
        "- Always use the 'pdf_extract' tool to extract relevant data to the user's query.\n"
        "- Pass the file located at the provided path  to the tool.\n"
        "- If the information extracted from the tool is irrelevant to the query, return: '----'"
        "- If the file path is invalid or the file cannot be processed, return : '------'"
        f"file path is : {file_path}"
        f"{suffix}"
    )

def make_internal_prompt(suffix: str) -> str:
    return (
        "answer the query of user"
        "if no relevant answer is found to the user's query, return the  message: '-----'"
        f"{suffix}"
    )


def make_searcher_prompt(suffix: str) -> str:
    return (
        "You are an independent web search agent tasked with extracting relevant information from web search."
        " You have access to two tools for this purpose:"
        "\n\n"
        "1. *google_search Tool*: Use this tool for queries related to stock market, finance, and real-time financial news."
        " It will provide information on stock prices, market trends, and other financial data from Google Finance."
        "\n\n"
        "2. *Tavily Tool*: Use this tool for all other general queries that are not specifically related to stock market or finance."
        " It is designed to help with various types of information across different domains."
        "\n\n"
        "Instructions:"
        "- For finance-related or stock market queries, you must use the *google_search Tool* to retrieve relevant stock market data or financial news."
        "- For all other queries, use the *Tavily Tool* to extract general information."
        "- If the information extracted from the tool(s) is irrelevant to the query, return the  message: '-----' "
        f"\n{suffix}"
    )
def make_astute_prompt(suffix: str,information_from_multiple_sources: str) -> str:
    return (
        f"""
You are an Astute Knowledge Consolidation Agent responsible for synthesizing information from the provided input and producing a reliable and concise answer.

### Context:
- You will receive all necessary information as a single string: information_from_multiple_sources.

### Instructions:
1. *Use Only the Provided Information*:
   - Do not source or refer to any additional data.
   - Rely exclusively on the content of information_from_multiple_sources.

2. *Process the Information*:
   - Always Pass the entire string {information_from_multiple_sources} into the iterative_consolidation_tool as initial_context to generate a consolidated version of the information.

3. *Deliver the Final Answer*:
   - Return the full output you got from tool from last 'iteration' , doesn't matter how lengthy.


"""
        + suffix
    )

def make_final_prompt(user_query: str, final_sub_responses: str) -> str:
    return f"""
You are a Financial Agent tasked with answering user queries using *only* the provided information.

### Instructions:
- Analyze the user's query: "{user_query}".
- Use *only* the information provided in: {final_sub_responses}.
- Do not rely on external knowledge, assumptions, or sources outside the given data.

### Rules:
1. If the information in {final_sub_responses} directly addresses the query, provide a clear and detailed answer.
2. If the information is incomplete or irrelevant to the query, acknowledge this honestly.
3. Include any numerical data, percentages, or financial terms from {final_sub_responses} that are relevant to the query.

### Response Format:
- Provide a *concise and accurate* response based strictly on {final_sub_responses}.
-Return Your answer
"""

from typing import Optional

def make_decider_agent_prompt(file_path: Optional[str] = None) -> str:
    # Remove the file_path_instruction entirely
    # and directly instruct the agent to call the 'generate_subquestions' tool
    return f"""
You are a Hybrid Intelligent Agent capable of responding to user queries with two distinct actions:

### Task:
1. *Handle Basic Queries*:
   - For queries that are simple, conversational, or general in nature and can be answered directly without tools, respond to the user immediately and concisely.
   - Examples of basic queries:
     - "What is the capital of France?"
     - "How's the weather today?" (if no real-time access is required)
     - "Tell me a joke."

2. *Delegate Complex Queries*:
   - If the query requires advanced reasoning, tool assistance, or additional processing (e.g., using provided data, generating sub-questions), pass the query to the 'generate_subquestions' tool to process the query. # Changed this line
   - The agent's job is only to call the tool and totally stop after calling it. The tool will handle everything required for the response.

### How to Decide:
- *Basic Query*: Can be answered with general knowledge, conversational ability, or straightforward logic.
- *Complex Query*: Requires access to tools, processing information, or generating sub-questions.

### Tool Access:
- You have access to the following tool:
  - 'generate_subquestions': Use this tool to break down complex queries into sub-questions and process information to generate a detailed response.

### Response Format:
1. *For Basic Queries*:
   - Respond directly with the prefix: *DIRECT RESPONSE:*
   - Example:

     DIRECT RESPONSE: The capital of France is Paris.


2. *For Complex Queries*:
   - Use the generate_subquestions tool with the full query or necessary context.
   - Totally stop after calling the tool.

### Guidelines:
- Avoid making assumptions beyond general knowledge for basic queries.
- Delegate complex queries without attempting to answer them yourself.

### Example Scenarios:
#### User Query: "What is 2 + 2?"
- *Action*: Respond directly.
- *Response*: "DIRECT RESPONSE: 2 + 2 equals 4."

#### User Query: "Analyze the impact of global warming on marine biodiversity."
- *Action*: Call generate_subquestions tool with the query.
- *Action Format*:
   - If file path provided: Include {file_path} in the tool invocation.
"""





def make_online_documents_retriever_prompt(suffix: str) -> str:
    return (
    f"""
You are an Online Documents Retriever Agent tasked with retrieving information relevant to a user's query from documents sourced online. You have access to two tools for this purpose:

### Workflow:
1. **Download Relevant Documents**:
   - Use the `downloader_tool` to download documents relevant to the user's query.
   - These documents will be saved in a folder named **'online_documents'**.

2. **Retrieve Similar Information**:
   - Use the `online_retriever_tool` to perform a similarity search across all documents in the **'online_documents'** folder.
   - Extract information related to the user's query based on embeddings and similarity search.

3. **Filter and Output Relevant Information**:
   - Analyze the extracted information to determine which parts are relevant to the user's query.
   - Provide a concise and accurate answer using only the relevant information.
   - If no relevant information is found, respond: **'No relevant information is found across the online documents.'**

### Instructions:
- Always use the two tools in the specified sequence:
  1. First call the `downloader_tool` to download documents.
  2. Then call the `online_retriever_tool` to search for relevant information.
- Do not fabricate or infer information not found in the retrieved documents.
- Ensure the final response is clear, concise, and directly addresses the user's query.
- If the user's query cannot be addressed using the downloaded documents, transparently communicate this.

### Tools:
1. **downloader_tool**:
   - Function: Downloads relevant documents from the web based on the query.
   - Input: User's query string.
   - Output: Confirmation of documents downloaded to the **'online_documents'** folder.

2. **online_retriever_tool**:
   - Function: Performs similarity search on the documents in the **'online_documents'** folder.
   - Input: User's query string.
   - Output: Extracted information relevant to the query, or confirmation if no relevant data is found.

### Response Format:
1. **For Successful Retrieval**:
   - Begin with the prefix: **RETRIEVED INFORMATION:**
   - Provide the relevant information extracted from the documents.
   - Example:
     ```
     RETRIEVED INFORMATION:
     - The impact of climate change includes rising sea levels and more frequent extreme weather events (Source: Document X).
     ```

2. **For No Relevant Information**:
   - Respond with: **'----'**

### Example Workflow:
#### User Query:
"What are the latest advancements in AI for healthcare?"

#### Agent Actions:
1. Call `downloader_tool` with query: "Latest advancements in AI for healthcare."
   - Response: "Documents have been downloaded to 'online_documents'."

2. Call `online_retriever_tool` with query: "Latest advancements in AI for healthcare."
   - Response: Extracted information from relevant documents.

3. Provide the final response:

"""
+ suffix
    )


online_documents_agent = create_react_agent(
llm,
tools=[downloader_tool,online_retriever_tool],
prompt=make_searcher_prompt("You can download the online documents and retrieve relevant information from them using the provided tools."))











from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langgraph.graph import MessagesState







search_agent = create_react_agent(
llm,
tools=[tavily_tool,google_search],
prompt=make_searcher_prompt("You can only search using the tools and return the answer"),

)

internal_agent = create_react_agent(
llm,
tools=[NoOpTool],
prompt=make_internal_prompt("you can only answer using the model's internal knowledge"),
)


## Block 2


from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain.tools import Tool

class FinancialAbbreviationTool:
    def __init__(self, document_path: str):
        self.document_path = document_path
        self.db = self._load_documents()

    def _load_documents(self):
        raw_documents = TextLoader(self.document_path).load()
        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
        documents = text_splitter.split_documents(raw_documents)
        return FAISS.from_documents(documents, embeddings)

    def retrieve_full_form(self, abbreviation: str) -> str:
        results = self.db.similarity_search(abbreviation)
        if results:
            return results[0].page_content  # Return the most relevant match
        return f"Full form for '{abbreviation}' not found."

abbreviation_tool = FinancialAbbreviationTool("/content/sample_data/Jargon_collection.txt")

retrieve_abbreviation_tool = Tool(
    name="RetrieveFullForms",
    func=abbreviation_tool.retrieve_full_form,
    description="Use this tool to retrieve the full form of abbreviations or shorthand terms in financial context."
)


def make_financial_agent_prompt(suffix: str) -> str:
    return f"""
    You are a Financial Context Agent responsible for analyzing and processing user queries effectively.

    ## Instructions:
    1. **Determine Context**: Analyze the query to identify its context (e.g., financial, legal, medical, general).
       - If the context is not financial, return the query unchanged.
    2. **Identify Abbreviations**: If the context is financial, detect any abbreviations or shorthand terms present in the query.
    3. **Retrieve Full Forms**: Use the 'RetrieveFullForms' tool to get the full forms of any abbreviations detected.
    4. **Augment Query**: Replace abbreviations in the query with their full forms to make it more precise and understandable.If not found full forms of any abbreviation ,let them remain same in query.
    5. **Final Response**: Return the final query **only**, without any tool usage logs or explanations.

    ## Strict Rules:
    - Do not include any tool usage logs, intermediate reasoning, or explanations in the output.
    - The output must be a plain string representing the final query only.
    - Any response containing metadata, tool invocation details, or explanations will be considered invalid.

    ## Query:
    {suffix}
    """

financial_agent = create_react_agent(
    llm,
    tools=[retrieve_abbreviation_tool],
    prompt=make_financial_agent_prompt("your task is to create the augmented query if requierd given theat the context of query is financial"),
)

stored_responses={}
string_representation=""

#### Block 1

import threading
import time
from pydantic import BaseModel, Field
from typing import List
from langchain.schema import SystemMessage, HumanMessage

# Define a pydantic model to enforce the output structure
class Questions(BaseModel):
    questions: List[str] = Field(
        description="A list of sub-questions related to the input query."
    )

final_sub_responses = ""

@tool
def generate_subquestions(query: str,file_path : str):
    """
    Tool to handle queries that require a complex retrieval.
    It can answer any question from simple to complex.

    """

    global final_sub_responses





    file_path=file_path
    # Define the system prompt dynamically based on the number of sub-questions
    system = f"""You are a helpful assistant designed to generate actionable sub-questions related to a given input query. Your primary goal is to break down complex queries into smaller, focused, and answerable sub-questions that can be independently addressed by LLMs or other tools.

### Instructions:
1. **Purpose**: Ensure each sub-question is specific and independently answerable to extract relevant information. The goal is to collect all necessary pieces of information for answering the user's main query.

2. **Relevance and Focus**:
   - Sub-questions must directly relate to the user's input query.
   - Avoid asking the user for additional information. Instead, focus on generating questions that the LLM or tools can answer using external data or documents.
   - Sub-questions should cover various aspects of the input query comprehensively but concisely.

3. **Avoid Asking Back**: Do not create sub-questions that ask the user for clarification or additional details. For example:
   - **Avoid**: "What specific financial metrics are you interested in?"
   - **Instead**: "What were Nykaa's total revenues in the last quarter of 2023?"

4. **Examples**:
   - Input Query: "Nykaa's last quarter financial report."
     - Sub-Questions:
       - "What was Nykaa's revenue in the last quarter of 2023?"
       - "What were Nykaa's expenses in the last quarter of 2023?"
       - "What was Nykaa's profit in the last quarter of 2023?"
       - "What is the overall performance of Nykaa in the last quarter of 2023?"

   - Input Query: "Impact of AI on healthcare."
     - Sub-Questions:
       - "What are the recent advancements in AI for disease detection?"
       - "How is AI used in personalized medicine?"
       - "What is the role of AI in drug discovery?"
       - "What are the benefits of AI in robotic surgeries?"
       - "What are the risks of using AI in healthcare?"

### Output Format:
- Generate sub-questions in a list format.
- Ensure the sub-questions are clear, concise, and directly actionable.
- Do not exceed 5 sub-questions.

"""

    # Initialize the structured model (example: a model capable of sub-question generation)
    structured_model = llm.with_structured_output(Questions)

    # Generate the sub-questions
    response = structured_model.invoke(
        [SystemMessage(content=system), HumanMessage(content=query)]
    )








    for question in response.questions:
      x=financial_agent.invoke({"messages":[HumanMessage(content=question)]})
      new_query= x["messages"][-1].content
      print("---------------")
      print(f"Subquery :    {new_query}")
      print("---------------")

      x=internal_agent.invoke({"messages":[HumanMessage(content=new_query)]})
      result=x["messages"][-1].content
      print("---------------")
      print(f"source1st : {result}")
      print("---------------")
      stored_responses['Source_1']=[result]


      x=search_agent.invoke({"messages":[HumanMessage(content=new_query)]})
      result=x["messages"][-1].content
      print("---------------")
      print(f"source2nd : {result}")
      print("---------------")
      stored_responses['Source_2']=[result]

      retriever_agent = create_react_agent(
      llm,
      tools=[extract_tool],
      prompt=make_retriever_prompt("You can only retrieve and answer from the file",file_path),
      )

      x=retriever_agent.invoke({"messages":[HumanMessage(content=new_query)]})
      result=x["messages"][-1].content
      print("---------------")
      print(f"source3rd : {result}")
      print("---------------")
      stored_responses['Source_3']=[result]






      # print(f"dictionary is : {stored_responses}")

      string_representation = '\n'.join(f"{key}: {value}" for key, value in stored_responses.items())

      # print(f"string is {string_representation}")


      astute_agent = create_react_agent(
      llm,
      tools=[iterative_consolidation_tool],
      prompt=make_astute_prompt(
          "Generate the final deliverable answer using the information received from the sources.",
          string_representation))

      x=astute_agent.invoke({"messages":[HumanMessage(content=new_query)]})
      result=x["messages"][-1].content
      final_sub_responses += result
      print("---------------")
      print(f"SUBANSWERS COLLECTIONS: {final_sub_responses}")
      print("---------------")


    x=online_documents_agent.invoke({"messages":[HumanMessage(content=query)]})
    print("----------------- Online agent content is ----")
    result=x["messages"][-1].content
    final_sub_responses += result
    print(f"source4th : {result}")
    print(f"SUBANSWERS COLLECTIONS FINAL: {final_sub_responses}")
    print("-----------------")


    final_agent=create_react_agent(
    llm,
    tools=[NoOpTool],
    prompt=make_final_prompt("Answer thoroughly the user's query using the provided information as string.",final_sub_responses),

)

    x=final_agent.invoke({"messages":[HumanMessage(content=query)]})

    print("---------------")
    print(query)
    final_result=x["messages"][-1].content
    print(f"Final Cumulative Answer: {final_result}")
    print("---------------")


    return ""








def run_function():

  from google.colab import files

  # Step 1: Ask the user if they want to upload a document
  user_input = input("Do you want to upload a document? (yes/no): ").strip().lower()

  # Step 2: Handle the user's choice
  if user_input in ['yes', 'y']:
      # Create the folder if it doesn't exist
      folder_name = 'input_documents'
      if not os.path.exists(folder_name):
          os.makedirs(folder_name)

      # Upload the file
      print("Please upload your document:")
      uploaded = files.upload()

      # Save the uploaded file to the folder and store the file path
      for filename in uploaded.keys():
          file_path = os.path.join(folder_name, filename)  # Construct the file path
          with open(file_path, 'wb') as f:
              f.write(uploaded[filename])  # Save the file to the folder
          print(f"File uploaded and saved at: {file_path}")
  else:
      print("No document uploaded.")



  decider_agent=create_react_agent(
      llm,
      tools=[generate_subquestions],
      prompt=make_decider_agent_prompt(file_path=None),


  )


  A=decider_agent.invoke({"messages":[HumanMessage(content=user_query)]})
  result=A["messages"][-1].content
  print(result)



#here query will be asked and fromhere the full model would work


user_query=input("Enter query please:")
if user_query:
  run_function()











